## Fine-tuning

In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# install trl for the SFT library
!pip install -q trl

# we need sentencepiece for the llama2 slow tokenizer
!pip install sentencepiece

In [3]:
from datasets import load_dataset
import json

dataset_name = 'annaluiza/MojoSnippets'
#data_files = {"train": "MojoTrain-Formatado.json", "test": "MojoTest-Formatado.json"}
data_files = {"train": "InstructTrain.jsonl", "test": "InstructTest.jsonl"}

dataset = load_dataset(dataset_name, data_files=data_files, trust_remote_code=True)

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/CodeLlama-7b-Instruct-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    use_flash_attention_2 = False, #set to True you're using A100
    device_map={"": 0}, #device_map="auto" will cause a problem in the training,
    #device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
import transformers

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,  use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
#data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [6]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 8

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)
model.add_adapter(peft_config)

In [7]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_8bit"
save_steps = 1
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=15,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to="none",
    logging_strategy="epoch",
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [23]:
eval_prompt = "Write a function to calculate the sum of two integers."

In [24]:
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], padding_side='left', skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a function to calculate the sum of two integers.    Args:        a: An integer.        b: Another integer.    Returns:        The sum of the two integers.    """    @staticmethod    @always_inline    fn sum(a: Int b: Int) -> Int:        return a + b    @staticmethod    fn test_sum() -> None:        var a = 1000000000000000000000000


In [11]:
from trl import SFTTrainer

max_seq_length = 256

trainer = SFTTrainer(
   model=model,
   train_dataset=dataset['train'],
   eval_dataset=dataset['test'],
   peft_config=peft_config,
   dataset_text_field="text",
   max_seq_length=max_seq_length,
   tokenizer=tokenizer,
   args=training_arguments,
   packing=False
   #data_collator=data_collator
)

In [12]:
import gc
import os
# attempting to not run out of memory
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()
gc.collect()

80

In [13]:
# train model
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# renable warnings
model.config.use_cache = True

Epoch,Training Loss,Validation Loss
0,2.867400,2.323521
2,1.756900,1.627810
4,1.319100,1.470379
6,0.942100,1.403423
8,0.680400,1.393963
10,0.519900,1.456941
12,0.395000,1.593452
14,0.272400,1.863892


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

## Avaliação do modelo

In [ ]:
from tensorboard import notebook
log_dir = "./logs"
notebook.start("--logdir {} --port 4000".format(log_dir))


## Examplos gerados

In [20]:
prompt = "Write a function to calculate the sum of two integers."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] Write a function to calculate the sum of two integers. [/INST]fn[    var[        a: Int[8]        b: Int[8]    ]    ]    return a + b[


In [21]:
prompt = "Write a function to calculate the multiplucation of two matrices."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] Write a function to calculate the multiplucation of two matrices. [/INST]fn[    alias[    alias[        "DType" = DType[        "DType" = "Float32"    ]    ]    "DType" = DType[    ]](    in_a: Tensor[    "DType"    ]    in_b: Tensor[    "DType"    ]    ) -> Tensor[    "DType"    ]:        var result = zeros[            in_a.shape[0]            in_b.shape[1]            "DType"        ]        for i in range(in_a.shape[0]):            for j in range(in_b.shape[1]):                result[i j] = in_a[i _] * in_b[j _]        return result[    ]    fn
